In [2]:
import requests
import os

def get_key_element():
    url = 'http://127.0.0.1:3000/'
    r = requests.get(url)
    return r.json()

In [3]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [4]:
def reduce_overlap(results, overlap_thresh=0.9):
    keys = list(results.keys())
    n = len(keys)
    keeps = []
    discards = []
    for a in range(n - 1):
        if keys[a] not in discards:
            keeps.append(keys[a])
            for b in range(a + 1, n):
                iou = bb_intersection_over_union(
                    list(results[keys[a]]['loc'].values()), 
                    list(results[keys[b]]['loc'].values())
                )
                if iou > overlap_thresh and keys[b] not in discards:
                    discards.append(keys[b])
                # break
        # break
    return keeps

In [5]:
import os
from PIL import Image, ImageDraw, ImageFont
import json

def run():
    font = ImageFont.truetype("LiberationSans-Regular.ttf", 10)

    cache_path = 'AngelHack_Group42_IntelligentImageAnalysis_Heineken/cache'
    img_path = os.listdir(cache_path)[0]
    ret_folder = img_path.split('.')[0]
    image = Image.open(
        os.path.join(
            cache_path,
            img_path
        )
    )
    key_element = get_key_element()

    ### PRODUCT RESULT
    key = 'product'
    results = key_element[key]
    im = image.copy()
    draw = ImageDraw.Draw(im)
    keeps = reduce_overlap(results)
    # print(keeps)
    for obj in keeps:
        loc = list(results[obj]['loc'].values())
        brand = results[obj]['brand']
        label = results[obj]['label']
        
        draw.rectangle(loc, width=5, outline=(255, 0, 0))
        draw.text(
            loc[:2], 
            f"{label}\n{brand}", 
            font=font, 
            fill=(1, 1, 1), 
            stroke_fill=(255, 255, 255),
            stroke_width=5
        )
    os.makedirs(f"eval_results/{ret_folder}", exist_ok=True)
    im.save(f"eval_results/{ret_folder}/{key}.jpg")

    ### HUMAN RESULT
    key = 'human'
    results = key_element[key]
    im = image.copy()
    draw = ImageDraw.Draw(im)
    # keeps = reduce_overlap(results)
    # print(keeps)
    for obj in results.keys():
        loc = list(results[obj]['loc'].values())
        gender = results[obj]['gender']
        emotion = results[obj]['emotion']
        action = results[obj]['action']
        role = results[obj]['role']
        
        draw.rectangle(loc, width=5, outline=(255, 0, 0))
        draw.text(
            loc[:2], 
            f"{gender}-{emotion}\n{action}-{role}", 
            font=font, 
            fill=(1, 1, 1), 
            stroke_fill=(255, 255, 255),
            stroke_width=5
        )
    os.makedirs(f"eval_results/{ret_folder}", exist_ok=True)
    im.save(f"eval_results/{ret_folder}/{key}.jpg")

    ### POSM RESULT
    key = 'posm'
    results = key_element[key]
    im = image.copy()
    draw = ImageDraw.Draw(im)
    keeps = reduce_overlap(results)
    for obj in keeps:
        loc = list(results[obj]['loc'].values())
        brand = results[obj]['brand']
        label = results[obj]['label']
        
        draw.rectangle(loc, width=5, outline=(255, 0, 0))
        draw.text(
            loc[:2], 
            f"{label}\n{brand}", 
            font=font, 
            fill=(1, 1, 1), 
            stroke_fill=(255, 255, 255),
            stroke_width=5
        )
    os.makedirs(f"eval_results/{ret_folder}", exist_ok=True)
    im.save(f"eval_results/{ret_folder}/{key}.jpg")

    ### CONTEXT CLASSIFY
    key = 'context'
    results = key_element[key]
    with open(f"eval_results/{ret_folder}/context.json", 'w') as f:
        json.dump(results, f)

In [8]:
import os
_path = '/home/yuuhanase/workspace/hackhcmc/hackhcmc_dataset'
_dest = "/home/yuuhanase/workspace/hackhcmc/AngelHack_Group42_IntelligentImageAnalysis_Heineken/AngelHack_Group42_IntelligentImageAnalysis_Heineken/cache"
files = os.listdir(_path)
for file in files[:30]:
    os.system(f"scp {_path}/{file} {_dest}")
    try:
        run()
    except:
        os.remove(f"{_dest}/{file}")